#### Step 1: Import Libraries

In [4]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.ensemble import RandomForestRegressor
import plotly.graph_objects as go
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


#### Step 2: Load and Preprocess Data

In [5]:
# Load the JSON file
data = []
with open('clean_trips.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

df = pd.DataFrame(data)

# Calculate trip time in seconds
df['trip_time'] = df['time']

# Drop rows with missing values (if any)
df.dropna(inplace=True)

#### Step 3: Prepare Features and Target

In [6]:
# Feature extraction functions
def extract_features(row):
    features = {}
    features['mean_time_gap'] = np.mean(row['time_gap'])
    features['std_time_gap'] = np.std(row['time_gap'])
    features['min_time_gap'] = np.min(row['time_gap'])
    features['max_time_gap'] = np.max(row['time_gap'])
    
    features['mean_lat'] = np.mean(row['lats'])
    features['std_lat'] = np.std(row['lats'])
    features['min_lat'] = np.min(row['lats'])
    features['max_lat'] = np.max(row['lats'])
    
    features['mean_lng'] = np.mean(row['lngs'])
    features['std_lng'] = np.std(row['lngs'])
    features['min_lng'] = np.min(row['lngs'])
    features['max_lng'] = np.max(row['lngs'])
    
    features['mean_dist_gap'] = np.mean(row['dist_gap'])
    features['std_dist_gap'] = np.std(row['dist_gap'])
    features['min_dist_gap'] = np.min(row['dist_gap'])
    features['max_dist_gap'] = np.max(row['dist_gap'])
    
    return pd.Series(features)

# Apply feature extraction to each row
feature_df = df.apply(extract_features, axis=1)

# Combine features with other relevant columns
X = pd.concat([feature_df, df[['driverID', 'weekID', 'timeID', 'dateID']]], axis=1)
y = df['trip_time']


#### Step 4: Split the Data

In [11]:
# Convert non-numeric columns to numeric or drop them
non_numeric_cols = X.select_dtypes(include=['object']).columns
X[non_numeric_cols] = X[non_numeric_cols].apply(lambda col: pd.factorize(col)[0])

# Ensure all features are numeric
assert X.select_dtypes(include=['object']).empty, "There are still non-numeric columns in X"

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

#### Step 5: Hyperparameter Tuning and Model Training

In [12]:
param_grid = {
    'n_estimators': list(range(1, 202, 50)),
    'max_depth': list(range(1, 8)),
    'min_samples_leaf': list(range(1, 10)),
    'max_leaf_nodes': [None] + list(range(10, 20, 2)),
    'random_state': [42],
    'criterion': ['squared_error', 'absolute_error']
}

grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5, refit=True, verbose=1, scoring='neg_root_mean_squared_error')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
model = RandomForestRegressor(**best_params)
model.fit(X_train, y_train)

Fitting 5 folds for each of 3780 candidates, totalling 18900 fits


#### Step 6: Evaluate the Model with Learning Curves

In [ ]:
train_sizes, train_scores, valid_scores = learning_curve(
    model, X_train, y_train, train_sizes=np.linspace(0.1, 1, 10), cv=5, scoring='neg_root_mean_squared_error')

fig = go.Figure()
fig.add_trace(go.Scatter(x=train_sizes, y=-np.mean(train_scores, axis=1), name='Training error'))
fig.add_trace(go.Scatter(x=train_sizes, y=-np.mean(valid_scores, axis=1), name='Validation error'))
fig.show()

#### Step 7: Make Predictions and Calculate Errors

In [ ]:
def estimations(test_data, model):
    test_data['trip_time_estimate'] = model.predict(test_data.drop(columns=['trip_time']))
    test_data['error'] = test_data['trip_time'] - test_data['trip_time_estimate']
    test_data = test_data.query("error < 3600")
    return test_data

plot_df = estimations(df.loc[X_test.index], model)

# Plotting and metrics functions can be reused as is
plot_graphs(plot_df)
get_metrics(plot_df)